In [2]:
################################
#### mit 1stNLPKurs Environment#
################################


import pandas as pd
import numpy as np
import sqlite3

# die Gratis api... funktioniert nur sporadisch...
#from googletrans import Translator
#from googletrans import LANGCODES
#languageCodes = LANGCODES
#translator = Translator()



import nltk
import spacy
nlp = spacy.load("de_core_news_sm")

from xml.etree import ElementTree as ET
import random
from random import *
import deepl

from sklearn.feature_extraction.text import CountVectorizer

import sklearn

from sklearn.model_selection import KFold as Kfold
from sklearn.model_selection import StratifiedKFold

import dill
import pickle

import os
from distutils.dir_util import copy_tree

import time


In [3]:
# für Aúthentifizierung an der Google API
# muss immer neu erstellt werden:
# https://console.cloud.google.com/iam-admin/serviceaccounts/details/110687257339442008990;edit=true?previousPage=%2Fapis%2Fcredentials%3Fproject%3Dalert-almanac-229709&project=alert-almanac-229709
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\tschu\\Downloads\\alert-almanac-229709-562a15e9cd9a.json"

from google.cloud import translate
from google.cloud import translate_v2 as translate
translate_client = translate.Client()

In [4]:
# open odeNet xml, parse it

#de_wn = open("2019-11-25_odenet-master\deWordNet.xml","r",encoding="utf-8")

de_wn = open("C:\\Users\\tschu\\OneDrive\\Master Data Science\\1_Master Thesis\\code_masterthesis\\The_final_November_Codings\\2019-11-25_odenet-master\\deWordNet.xml","r",encoding="utf-8")

out_lex = open("out_lex.txt","w",encoding="utf-8")

tree = ET.parse(de_wn)

root = tree.getroot()

lexicon = root.find('Lexicon')

In [5]:
def check_word_lemma(word_to_check):
    
    """
    greift auf Lexikon Einträge zu, bekommt Lexikon-ID für ein Wort,
    den Lemma-Wert
    """
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        lemma_id = lexentry.attrib['id']
        if lemma_value == word_to_check:
            pos = lemma.attrib['partOfSpeech']
            senses = []
            for sense in lexentry.iter('Sense'):
                sense_id = sense.attrib['id']
                synset_id = sense.attrib['synset']
#                senserelation_type = lexentry.find('SenseRelation').attrib['relType']
#                senserelation_target = lexentry.find('SenseRelation').attrib['target']
                #senses.append([sense_id,synset_id])
                senses.append(synset_id)
#            print("LEMMA: " + lemma_value + "\nPOS: " + pos + "\nSENSE ID: " + sense_id)
            #return(lemma_id, lemma_value, pos, senses)
            return(senses)

#wir brauchen ja nur den sense...

In [6]:
def words_in_synset(id):
    words = []
    for lexentry in lexicon.iter('LexicalEntry'):
        for sense in lexentry.iter('Sense'):
            if sense.attrib['synset'] == id:
                lemma = lexentry.find('Lemma').attrib['writtenForm']
                words.append(lemma)
    return(words)

In [7]:
def find_syno(post):
    
    """
    rufe das mit einem Post aus der Liste auf. 
    liefert die Synonyme und den dazugehörigen Text
    
    """

    o_words = []
    s_words = [] 
    fuu = [] # brauch ich doch, sonst problem wennn leer. 
    final_sätze = []
    unaugmentierbare_sätze = []
    
    

    doc =nlp(post)
    for token in doc:
        if token.pos_== "NOUN":
            #o_words.append(token.text)
            lemma = check_word_lemma(token.text)
            if lemma is None:
                continue 
            else:
                o_words.append(token.text)
                z = words_in_synset(lemma[0])
                try:
                    s_words.append(z[1])
                except IndexError: 
                    pass
                fuu = dict(zip(o_words, s_words))
                satz_to_mod = doc.text
                #print(satz_to_mod)
                

    if len(fuu) == 0:
        unaugmentierbare_sätze.append(doc.text)
        pass
        
    else:  
        for k,v in fuu.items():
            satz_to_mod = satz_to_mod.replace(k,v)
        final_sätze.append(satz_to_mod) # DIE EINRÜCKUNG WARS GEWESEN ;-)
 

              
        
        
    #return fuu, doc.text   
    return final_sätze, unaugmentierbare_sätze

# raised exception wenn keine synonyme... fuu 

In [9]:
#diskrim = pd.read_csv("2019-12-02-clean_files_for_Aug\diskrim_corr.tsv", sep="\t")
#perso = pd.read_csv("2019-12-02-clean_files_for_Aug\perso_corr.tsv", sep="\t")
#no_lesen_fb = pd.read_csv("2019-12-02-clean_files_for_Aug\Kann_ni_gelesen_werden_fb_corr.tsv", sep="\t") # wenn file feedback heisst, kann es nicht geladen werden, wird nicht gefunden...
#sepos=pd.read_csv("C:\\Users\\tschu\\OneDrive\\Master Data Science\\1_Master Thesis\\code_masterthesis\\The_final_November_Codings\\2019-12-02-clean_files_for_Aug\\sepos_corr.tsv", sep="\t")
#seneg=pd.read_csv("2019-12-02-clean_files_for_Aug\seneg_corr.tsv", sep="\t")
#seneu=pd.read_csv("C:\\Users\\tschu\\OneDrive\\Master Data Science\\1_Master Thesis\\code_masterthesis\\The_final_November_Codings\\2019-12-02-clean_files_for_Aug\\seneu_corr.tsv", sep="\t")
#offto=pd.read_csv("2019-12-02-clean_files_for_Aug\offto_corr.tsv", sep="\t")
#inapp=pd.read_csv("C:\\Users\\tschu\\OneDrive\\Master Data Science\\1_Master Thesis\\code_masterthesis\\The_final_November_Codings\\2019-12-02-clean_files_for_Aug\\inapp_corr.tsv", sep="\t")
no_lesen_ar= pd.read_csv("C:\\Users\\tschu\\OneDrive\\Master Data Science\\1_Master Thesis\\code_masterthesis\\The_final_November_Codings\\2019-12-02-clean_files_for_Aug\\Kann_ni_gelesen_werden_ar_corr.tsv", sep="\t")

In [10]:
# wird belassen sepos aber mit seneutral gefüllt ;-)

seposAug = no_lesen_ar[no_lesen_ar["label"]==1]

In [11]:
len(seposAug)

1013

In [12]:
def augSyno(df):

# Augmentiert mit Synony-Replacement, erstellt eine Liste mit jeweils augmentierten und anaugemntierten Posts

    augmentiert = []
    unaugmentiert = []
    
    for i in df["text"]:
        aug,unaug =find_syno(i)
        augmentiert.append(aug)
        unaugmentiert.append(unaug)
        
    return augmentiert, unaugmentiert

In [13]:
aug, unaug = augSyno(seposAug)

In [14]:
#Augmentierte Sätze werden der Liste hinzugefügt, unaugmentierte als []
count = 0
for i in aug:
    if len(i) == 1:
        count += 1
print(count, "Sätze konnten augmentiert werden")

871 Sätze konnten augmentiert werden


In [15]:
count = 0
for i in unaug:
    if len(i) == 1:
        count += 1
print(count, "Sätze konnten nicht augmentiert werden")

142 Sätze konnten nicht augmentiert werden


In [16]:
seposAug["Syno-1"] = aug

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
seposAug['Syno-1'] = seposAug['Syno-1'].str[0]

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
def transi(post):
    
    
    """
    Rückübersetzung mit Google, kann wengen des Schnitstellen Limits nicht eingesetzt werden.
    en = translator.translate(post, src="de",dest="en")
    de_back = translator.translate(en.text, src="en",dest="de")
    de_back = de_back.text
    
    """
    
    resultEN = translate_client.translate(post, target_language="en")
    resultDE = translate_client.translate(resultEN["translatedText"], target_language="de")
    de_back = resultDE["translatedText"]
    
    

    return de_back

In [20]:
# Die Posts, welche im ersten Schritt durch Synonym Replacement nicht augmentiert werden konnten
# werden mit Google (kostenpflichitg) rückübersetzt. 

backtransGoogle = []

for i in unaug:
    if len(i) == 0:
        #backtransi.append([]) # cannot backtranslate list.. 
        backtransGoogle.append("")
    else:
        translated = transi(i[0])
        time.sleep(1)
        
        backtransGoogle.append(translated)

In [21]:
# jetzt für die rückübersetzten wieder Synonyme finden


aug1lst = []
unaug1lst = []

for i in backtransGoogle:
    aug1,unaug1 =find_syno(i)
    aug1lst.append(aug1)
    unaug1lst.append(unaug1)

In [22]:
count = 0
for i in aug1lst:
    if len(i) == 1:
        count += 1
print(count, "Sätze konnten augmentiert werden")

106 Sätze konnten augmentiert werden


In [23]:
seposAug["Syno-backtrans"] = aug1lst

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
seposAug['Syno-backtrans'] = seposAug['Syno-backtrans'].str[0]

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
seposAug.head()

,id,label,alpha,text,Syno-1,Syno-backtrans
1,1,1,a,Gott sei dank ist für sie eine Umfrage alles ...,Der Ewige sei dank ist für sie eine Erkundigu...,NaN
2,2,1,a,Die FPÖ wird aus allen Rohren schießen und di...,NaN,NaN
9,9,1,a,Es sind immer dieselben bigotten verlogenen Be...,Es sind immer dieselben bigotten verlogenen Be...,NaN
11,11,1,a,ich sag jetzt mal Sie sind nicht das Problem d...,ich sag jetzt mal Sie sind nicht das Kiste das...,NaN
12,12,1,a,nein sie sind nicht im parlament vertreten. d...,nein sie sind nicht im parlament vertreten. d...,NaN


In [27]:
seposAug["mergedSy-be"] = seposAug['Syno-1'].fillna(seposAug['Syno-backtrans'])

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
# Sätze konneten nicht augmentiert werden
seposAug["mergedSy-be"].isna().sum()

36

In [30]:
from yandex.Translater import Translater
tr = Translater()
tr.set_key('trnsl.1.1.20191205T025116Z.7d58d3f4e20269f5.23ea844013c725adfd0e9370c3a3764b8507d8bd') # Api key found on https://translate.yandex.com/developers/keys


In [46]:
yannitrans2 = []

"""
Führt Rückübersetzung der Originalposts über das Russische durch. 

Yandex API
the volume of the text translated: 1,000,000 characters per day but not more than 10,000,000 per month.
stayblue-ie, das mti der Sonne
https://translate.yandex.com/developers/keys
Flo Underwayyer

API Key for project-augmentation

trnsl.1.1.20191205T025116Z.7d58d3f4e20269f5.23ea844013c725adfd0e9370c3a3764b8507d8bd

ist die Million charakters aufgebraucht, gibts forbidden Error. 

"""



for i in postforYanniCut["text"]: #
    #tr.set_key('trnsl.1.1.20191205T025116Z.7d58d3f4e20269f5.23ea844013c725adfd0e9370c3a3764b8507d8bd') # Api key found on https://translate.yandex.com/developers/keys
    tr.set_from_lang('de')
    tr.set_to_lang('ru')
    ru = tr.set_text(i)
    trans = tr.translate()
    
    tr.set_from_lang('ru')
    tr.set_to_lang('de')
    de = tr.set_text(trans)
    backtrans = tr.translate()
    
    yannitrans2.append(backtrans)

In [47]:
len(yannitrans2)

127

In [35]:
yannitrans[885]

' Ja, und jetzt allen ernstes der Rechtfertigung für den Mord komme zufällig vorbei stehender Passanten mit einem Messer? Auf dem Video nicht zu sehen was der Junge getan hat früher. Misshandlung seitens der Grenz-Polizei kann man nicht sprechen in jedem Fall. Da kommt Folgendes viel eher als Ursache in Frage '

In [40]:
seposAug["text"].iloc[885]

' Aha und das ist jetzt allen Ernstes Ihre Rechtfertigung für die Ermordung zufällig vorbeikommender Passanten mit dem Messer? Auf dem Video ist nicht zu sehen was der Junge vorher gemacht hat. Von Misshandlung durch die Grenzpolizisten kann man jedenfalls nicht sprechen. Da kommt folgendes viel eher als Ursache in Frage '

In [78]:
len(yannitrans2)

384

In [48]:
yan = yannitrans + yannitrans2

In [49]:
len(yan)

1013

In [72]:
yannitrans[1445]

'verknüpfen ich Versuch mal enden, vielleicht, Thomas Leopold würde nicht machen diese täglichen Erfahrungen in einem solchen Ausmaß er bereits früher solche Projekte geben würde, vielleicht mit einer großen Auswahl, sondern auch für Kinder türkischer Einwanderer. Integration will wissen, es ist nicht immer sicher geben, aber in einem solchen Ausmaß, wie wir es jetzt sehen. ich hoffe, dass ein solcher Ansatz wird einen wichtigen Beitrag zur Integration und auch weiter finanziert werden.'

In [60]:
seposAug["text"].iloc[1445]

'ich probier mal die enden zu verknüpfen vielleicht würde thomasleopold diese täglichen erfahrungen nicht in diesem umfang machen hätte es früher schon derartige projekte gegeben vielleicht auch mit mehr reichweite und auch für kinder von türkischen immigranten. integrationsunwillige wirds sicher immer geben aber nicht in dem ausmaß wie wir es momentan sehen. ich hoffe dass dieser ansatz ein wichtiger baustein in der integration wird und auch weiter finanziert wird.'

In [44]:
postforYanniCut = seposAug.iloc[886: 1013]

In [55]:
len(seposAug)

1830

In [52]:
seposAug["text"].iloc[1829] # letzte Nummer 

' Was? Unsinn! Der Linguistik turn beschränkt sich auf Metaphysische Realitäten ... so wie die realität des wortes rot . Und aus sicht von Philosophen die den mit gemacht haben macht der Satz ...dass nicht mehr die realen Tatsachen zu realen Einstellungen führen nein die falschen Wörter führen zu den falschen Einstellungen gar keinen Sinn. Was sie meinen ist ein metaphysische Position... nämlich die der Antirealisten oder Konstruktivsten... aber auch nur in der Bla bla form der Politisierung von Leuten die Genderstudies machen.'

In [56]:
seposAug

,id,label,alpha,text,Syno-1,Syno-backtrans,mergedSy-be
0,0,1,a,Top qualifizierte Leute verdienen auch viel.,Top qualifizierte Bruder verdienen auch viel.,NaN,Top qualifizierte Bruder verdienen auch viel.
2,2,1,a,Die FPÖ wird aus allen Rohren schießen und di...,NaN,NaN,NaN
4,4,1,a,Na wer weis was da vorgefallen ist...,NaN,NaN,NaN
6,6,1,a,Juhu ich werde reich! Very Urgent Bastard l am...,Juhu ich werde reich! Very Urgent Bastard l am...,NaN,Juhu ich werde reich! Very Urgent Bastard l am...
7,7,1,a,was die meise gibt kein black metal konzert?,NaN,NaN,NaN
8,8,1,a,Musst nicht sonst gäbs weniger Spaß!,Musst nicht sonst gäbs weniger Entzücken!,NaN,Musst nicht sonst gäbs weniger Entzücken!
11,11,1,a,ich sag jetzt mal Sie sind nicht das Problem d...,ich sag jetzt mal Sie sind nicht das Kiste das...,NaN,ich sag jetzt mal Sie sind nicht das Kiste das...
13,13,1,a,Dann haben Sie mein Kommentar absolut richtig...,Dann haben Sie mein Anmerkung absolut richtig...,NaN,Dann haben Sie mein Anmerkung absolut richtig...
14,14,1,a,Natürlich trifft eine hohe Mehrwertsteuer in ...,Natürlich trifft eine hohe Mehrwertsteuer in ...,NaN,Natürlich trifft eine hohe Mehrwertsteuer in ...
16,16,1,a,Dass Schäuble am Wenigsten versteht ist eine ...,Dass Schäuble am Wenigsten versteht ist eine ...,NaN,Dass Schäuble am Wenigsten versteht ist eine ...


In [31]:
# abgerochene Liste Speichern, wird morgen weiteraugmentiert 
file = open("seneuYannitrans.txt", "wb")
pickle.dump(yannitrans, file)
file.close()

In [50]:
seposAug["ru-back-yandex"] = yan

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
seposAug = seposAug.drop(columns=["Syno-1","Syno-backtrans"])

In [53]:
seposAug["mergedSy-be"].isna().sum()

36

In [54]:
seposAug = seposAug.rename(columns={"mergedSy-be": "text_syn"})

In [56]:
# Achtung
nosepos = no_lesen_ar[no_lesen_ar["label"]==0]

In [57]:
len(nosepos)

2537

In [58]:
nosepos["text_syn"] = np.nan

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
nosepos["ru-back-yandex"] = np.nan

c:\users\tschu\.conda\envs\1stnlpkurs\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
nosepos.head()

,id,label,alpha,text,text_syn,ru-back-yandex
0,0,0,a,Top qualifizierte Leute verdienen auch viel.,NaN,NaN
3,3,0,a,Weil es dein meisten Leuten verständlicherwei...,NaN,NaN
4,4,0,a,Na wer weis was da vorgefallen ist...,NaN,NaN
5,5,0,a,Gabalier? Künstler? Bruahahaha!,NaN,NaN
6,6,0,a,Juhu ich werde reich! Very Urgent Bastard l am...,NaN,NaN


In [61]:
frames = [nosepos, seposAug]

In [62]:
finaldf = pd.concat(frames)

In [64]:
finaldf = finaldf.reset_index(drop = True)

In [65]:
#rewrite the id
finaldf["id"] = finaldf.index

In [66]:
finaldf

,id,label,alpha,text,text_syn,ru-back-yandex
0,0,0,a,Top qualifizierte Leute verdienen auch viel.,NaN,NaN
1,1,0,a,Weil es dein meisten Leuten verständlicherwei...,NaN,NaN
2,2,0,a,Na wer weis was da vorgefallen ist...,NaN,NaN
3,3,0,a,Gabalier? Künstler? Bruahahaha!,NaN,NaN
4,4,0,a,Juhu ich werde reich! Very Urgent Bastard l am...,NaN,NaN
5,5,0,a,was die meise gibt kein black metal konzert?,NaN,NaN
6,6,0,a,Musst nicht sonst gäbs weniger Spaß!,NaN,NaN
7,7,0,a,du kannst sie auch ausdrucken und unters Kopf...,NaN,NaN
8,8,0,a,so ein psycho,NaN,NaN
9,9,0,a,leider hat keine Band so schwer abgebaut wie ...,NaN,NaN


In [67]:
# mit dem kann dann 10 Fold Split gemacht werden....

file = open("ar_corr/ar_corrAug.txt", "wb")
pickle.dump(finaldf, file)
file.close()